In [ ]:
import operator
filename = "input_24"
#filename = "example_24.txt"
with open(filename) as f:
    input1, input2 = f.read().split("\n\n")

## Part 1:

In [2]:
operator_map = {"AND": operator.__and__,
                "OR": operator.__or__,
                "XOR": operator.__xor__}

In [3]:
input_wires = dict([(el[0:3],int(el[-1])) for el in input1.split("\n")])
gates = [line.split(" ") for line in input2.split("\n")]
wires = dict([(gate[0],None) for gate in gates] + 
             [(gate[2],None) for gate in gates] + 
             [(gate[4],None) for gate in gates]) | input_wires

In [4]:
while gates:
    for gate in gates:
        try:
            x = operator_map[gate[1]](wires[gate[0]], wires[gate[2]])
        except TypeError: pass
        else:
            wires[gate[4]] = x
            gates.remove(gate)

In [5]:
output = ""
for i in range(100):
    number = "z" + str(i).zfill(2)
    try:
        output += str(wires[number])
    except KeyError:
        break

In [ ]:
int(output[::-1], base=2)

## Part 2:

Every operation on input wires xi, yi (starting with x02) has the following form: \
xi XOR yi = aaa --> aaa XOR previous_carry = zi \
                    aaa AND previous_carry = ccc \
xi and yi = bbb --> bbb OR ccc = carry \
Where previous_carry is the value of carry from the last operation.

In [51]:
def swap_gates(gates, problem_gates):
    gate1, gate2 = problem_gates
    output1 = gates[gate1]
    output2 = gates[gate2]
    gates[gate2] = output1
    gates[tuple(reversed(gate2))] = output1
    gates[gate1] = output2
    gates[tuple(reversed(gate1))] = output2
    return gates

In [52]:
gates_list = [line.split(" ") for line in input2.split("\n")]
gates = {}
for gate in gates_list:
    # add gate partners in both orders
    gates[tuple(gate[:3])] = gate[4]
    gates[tuple(reversed(gate[:3]))] = gate[4]

In [53]:
faulty_gates = []
carry = "htw"
for i in range(2, len(input_wires) // 2):
    x = "x" + str(i).zfill(2)
    y = "y" + str(i).zfill(2)
    z = "z" + str(i).zfill(2)

    while True:
        problem_gates = set()

        x_xor_y = gates[(x, "XOR", y)]
        if x_xor_y == z:
            problem_gates.add((x, "XOR", y))
        
        x_and_y = gates[(x, "AND", y)]
        if x_and_y == z:
            problem_gates.add((x, "AND", y))

        try: 
            output = gates[(x_xor_y, "XOR", carry)]
            if output != z:
                problem_gates.add((x_xor_y, "XOR", carry))
        except KeyError:
            problem_gates.add((x, "XOR", y))
        if len(problem_gates) == 2:
            gates = swap_gates(gates, list(problem_gates))
            faulty_gates.extend(problem_gates)
            continue

        try:
            intermediate = gates[(x_xor_y, "AND", carry)]
            if intermediate == z:
                problem_gates.add((x_xor_y, "AND", carry))
        except KeyError:
            problem_gates.add((x, "XOR", y))
        if len(problem_gates) == 2:
            gates = swap_gates(gates, list(problem_gates))
            faulty_gates.extend(problem_gates)
            continue

        try:
            carry = gates[(x_and_y, "OR", intermediate)]
            if carry == z:
                problem_gates.add((x_and_y, "OR", intermediate))
        except KeyError:
            problem_gates.add((x, "AND", y))
        if len(problem_gates) == 2:
            gates = swap_gates(gates, list(problem_gates))
            faulty_gates.extend(problem_gates)
            continue

        if len(problem_gates) < 2:
            break


In [ ]:
faulty_gates

In [ ]:
faulty_nodes = [gates[gate] for gate in faulty_gates]
print(",".join(sorted(faulty_nodes)))

+ first pair: x10 and y10 = z10, kck xor skm = vcf
+ second pair: qjg and jjf = z17, qjg xor jjf = fhg
+ third pair: x35 and y35 = dvb, x35 xor y35 = fsq
+ fourth pair: rvd or wrj = z39, kmh xor mnd = tnc

## Visualization

In [63]:
import networkx as nx
import matplotlib.pyplot as plt

In [67]:
G = nx.Graph()
gates = [line.split(" ") for line in input2.split("\n")]

# add input nodes:
G.add_nodes_from(input_wires.keys())
# add intermediate nodes:
for i, gate in enumerate(gates):
    G.add_node(gate[0])
    G.add_node(gate[2])
    G.add_node(gate[4])
    G.add_node((gate[1],i))

In [68]:
# add edges:
for i, gate in enumerate(gates):
    G.add_edge(gate[0], (gate[1],i))
    G.add_edge(gate[2], (gate[1],i))
    G.add_edge((gate[1],i), gate[4])

In [ ]:
ax = plt.figure(figsize=(30,30))
nx.draw(G, with_labels=True, font_weight='bold')